In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import tensorflow_recommenders as tfrs

<h2>Data preparation</h2>

For this part we need to pre process the data and then convert it into a tf.data.Dataset


In [144]:
import pandas as pd
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')
#remouve nan values
anime = anime.dropna()
rating = rating.dropna()


rating['user_id'] = rating['user_id'].astype(str)
rating['anime_id'] = rating['anime_id'].astype(str)
anime["anime_id"] = anime["anime_id"].astype(str)

unique_user_ids = list(rating.user_id.unique())

unique_anime_ids = list(anime.anime_id.unique())

In [145]:

unique_user_ids = np.unique(unique_user_ids)
unique_anime_ids = np.unique(unique_anime_ids)

In [146]:
ratings = tf.data.Dataset.from_tensor_slices(dict(rating))
ratings 

<_TensorSliceDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'anime_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [147]:
ratings.dtype

AttributeError: '_TensorSliceDataset' object has no attribute 'dtype'

In [148]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)


In [149]:
train

<_TakeDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'anime_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<h2>Model definition</h2>

In [150]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.anime_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_anime_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_anime_ids) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, anime_id = inputs
    
    """user_id=tf.convert_to_tensor(user_id)
    anime_id=tf.convert_to_tensor(anime_id)"""
    
    user_embedding = self.user_embeddings(user_id)
    anime_embedding = self.anime_embeddings(anime_id)

    return self.ratings(tf.concat([user_embedding, anime_embedding], axis=1))

In [167]:
model = RankingModel()

model.call((tf.convert_to_tensor(tf.convert_to_tensor(['1'])), tf.convert_to_tensor(['32281'])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.0285008]], dtype=float32)>

In [152]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [184]:
class AnimeModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["anime_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    """labels = features.pop("rating")

    rating_predictions = self(features)
    """
    features = features.copy()  # Make a shallow copy of features
    labels = features.pop("rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [185]:
model = AnimeModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [186]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()


In [188]:
model.fit(cached_train, epochs=100)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 5.6920 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.4293 - total_loss: 5.6920
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 5.0316 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.2672 - total_loss: 5.0316
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.7583 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.1992 - total_loss: 4.7583
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.5929 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.1584 - total_loss: 4.5929
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.4828 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.1318 - total_loss: 4.4828
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.3999 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.1113 - total_loss: 4.3999
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.3614

In [189]:
model.evaluate(cached_test, return_dict=True)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 7.0060 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.5220 - total_loss: 7.0060


{'loss': 7.782638072967529,
 'regularization_loss': 0.0,
 'root_mean_squared_error': 2.6130995750427246,
 'total_loss': 7.782638072967529}

In [190]:
test_ratings = {}
test_movie_titles = ["20","154","170"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": tf.convert_to_tensor(["3"]),
      "anime_id": tf.convert_to_tensor([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
170: [[9.021084]]
154: [[7.4876933]]
20: [[7.288632]]


In [ ]:
#2345